## Exercícios

> Retirados de [learn-python: sqlalchemy_orm-questions](https://aviadr1.github.io/learn-advanced-python/11_db_access/exercise/sqlalchemy_orm-questions.html).

#### Q1.

Baixa e extraia o arquivo compactado com o banco de dados [Chinook database](https://www.sqlitetutorial.net/sqlite-sample-database/). Salve o arquivo `chinook.db` na mesma pasta deste script.
* Link para baixar: http://www.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip

<img width=500 src=https://www.sqlitetutorial.net/wp-content/uploads/2015/11/sqlite-sample-database-color.jpg>


#### Q2.

Leia o código e os comentários das células a seguir para entender como acessamos os modelos ORM de um banco já existente.

In [26]:
from sqlalchemy import create_engine, text, MetaData
from sqlalchemy.orm import Session

engine = create_engine("sqlite+pysqlite:///chinook.db", echo=False)

### extrai as classes da base de dados Chinook
metadata = MetaData()
metadata.reflect(engine)

# O metadata tem informações sobre as tabelas
# que serão usadas para criar os modelos ORM
for table_name, table in metadata.tables.items():
    print(table_name)
    print(table.columns.keys())
    print(table.columns.items())
    print('-'*25)

### configura o objeto Base mapeando os modelos ORM das tabelas
from sqlalchemy.ext.automap import automap_base
Base = automap_base(metadata=metadata)
Base.prepare()

# o objeto Base tem os modelos ORM que podemos usar
# para manipular o banco de dados
print(Base.classes.items())


[]


#### Q3.
Com base nos códigos anteriores realize as operações solicitadas nas células a seguir:


In [24]:
tracks = session.scalars(select(Base.classes.Track).limit(3)).all()
for t in tracks:
    print(t.TrackId, t.Name, t.AlbumId)

AttributeError: Track

In [25]:
stmt = (
    select(Track.Name, Album.Title)
    .join(Album, Track.AlbumId == Album.AlbumId)
    .limit(20)
)
res = session.execute(stmt).all()
for name, album in res:
    print(name, "-", album)


NameError: name 'Track' is not defined

In [ ]:
items = session.scalars(select(InvoiceItem).limit(10)).all()
for it in items:
    print(it.InvoiceLineId, it.TrackId, it.Quantity)


In [ ]:
stmt = (
    select(Track.Name, InvoiceItem.Quantity)
    .join(Track, Track.TrackId == InvoiceItem.TrackId)
    .limit(10)
)
res = session.execute(stmt).all()
for name, qty in res:
    print(name, "-", qty)


In [ ]:
stmt = (
    select(Artist.Name, func.sum(InvoiceItem.Quantity).label("total_sold"))
    .join(Album, Artist.ArtistId == Album.ArtistId)
    .join(Track, Album.AlbumId == Track.AlbumId)
    .join(InvoiceItem, Track.TrackId == InvoiceItem.TrackId)
    .group_by(Artist.ArtistId)
    .order_by(func.sum(InvoiceItem.Quantity).desc())
    .limit(10)
)
res = session.execute(stmt).all()
for artist, total in res:
    print(artist, "-", total)
